# Google Review에서 동반자 유형 빈도수 추출 코드 

* 구글 리뷰 데이터셋 파일들(각 관광지별로 저장된 15개의 파일들)이 있는 폴더를 타겟으로 지정한다. 
* 동반자 유형 사전을 만든다. 
* 구글 리뷰에 대해 특정 조사로 끝나는 단어들만 정제한다. 
* 품사 태깅을 진행하여 명사와 조사를 추출한다. 
* 명사가 동반자 유형 사전에 해당하는지 확인후 카운팅 한다. 
* 빈도수가 높은 순서대로 동반자 유형을 출력한다. 
* 해당 과정을 각 관광지별로 진행하고, 이후 전체 관광지 15개를 대상으로도 진행한다. 

## googleReviewComparison 코드 문제점 

* 추출된 키워드를 확인했을때 동반자 유형이라 볼 수 없는 경우도 카운팅 된다. 
    - Komoran과 Kahiii를 성능 확인 후 Kahiii를 Stemming에 적용하여 동사 원형 사전을 추가해 방문과 직접적인 관련이 있는 동사가 포함된 문장만 동반자 유형 빈도수 카운팅에 반영한다.
* 추출된 키워드 개수를 셀 때 같은 key를 가진 경우가 여러번 카운팅되는 문제점이 있다. 
    - Set을 통해 중복으로 나타난 key를 정제한다. 

## 타겟 파일 설정

In [1]:
target = 'labeled_spellcheck_8420'

## 사용 라이브러리 

In [2]:
import os # file path 
import glob # file path 
from tqdm import tqdm # execution time
import re # regex
import pandas as pd # dataframe
import copy # deep copy

from collections import Counter
from ckonlpy.tag import Twitter
from ckonlpy.tag import Postprocessor

In [3]:
import warnings

warnings.filterwarnings( 'ignore' )

## 경로 설정

In [4]:
!tree -L 3 -N /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420 # jungcheck 폴더내 filtered(전처리 후)의 파일트리 확인

/home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420
├── google_reviews_art_culture_complex.csv
├── google_reviews_daecheong_lake.csv
├── google_reviews_dongchundang.csv
├── google_reviews_expo_science_park.csv
├── google_reviews_gyejok_mountain.csv
├── google_reviews_hanbat_arboretum.csv
├── google_reviews_jangtae_mountain.csv
├── google_reviews_observatory.csv
├── google_reviews_oworld_zoo.csv
├── google_reviews_ppuri_park.csv
├── google_reviews_science_museum.csv
├── google_reviews_sungsimdang_bakery.csv
├── google_reviews_uineungjeongi_street.csv
├── google_reviews_water_barrel.csv
└── google_reviews_yuseong_hotspring.csv

0 directories, 15 files


In [5]:
!pwd

/home/aiffel-dj19/jungcheck/EDA


In [6]:
g_data_path = '/home/aiffel-dj19/jungcheck/DataPreprocessing'

In [7]:
filtered_path = os.path.join(g_data_path, 'filtered/google_review/')
filtered_path

'/home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/'

In [8]:
filtered_fds = os.listdir(filtered_path) # fds: 'folders'
print(f'{filtered_path}내의 파일 개수: {len(filtered_fds)}개') 
print()
print(filtered_fds)

/home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/내의 파일 개수: 10개

['google_reviews_spacing_8421.csv', 'labeled_spacing_8421', 'spacing_8421', 'google_reviews_labeled_spellcheck_8420.csv', 'labeled_spellcheck_8420', 'spacing_spellcheck_8420', 'spellcheck_8420', 'google_reviews_nolabel_8429.csv', 'google_reviews_labeled_spacing_8421.csv', 'google_reviews_labeled_8429.csv']


In [9]:
# english keyword to korean keyword
eng2kor = {
            'jangtae_mountain': '장태산',
             'gyejok_mountain': '계족산',
             'dongchundang': '동춘당',
             'uineungjeongi_street': '으느정이문화의거리',
             'ppuri_park': '뿌리공원',
             'expo_science_park': '엑스포',
             'sungsimdang_bakery': '성심당',
             'water_barrel': '수통골',
             'yuseong_hotspring': '유성온천',
             'hanbat_arboretum': '한밭수목원',
             'science_museum': '국립중앙과학관',
             'daecheong_lake': '대청호',
             'art_culture_complex': '대전문화예술단지',
             'observatory': '시민천문대',
             'oworld_zoo': '오월드'
}

# korean keyword to english keyword
kor2eng = {v:k for k, v in eng2kor.items()}

print('영어 keyword => 한국어 keyword')
print(eng2kor)
print()
print('한국어 keyword => 영어 keyword')
print(kor2eng)

영어 keyword => 한국어 keyword
{'jangtae_mountain': '장태산', 'gyejok_mountain': '계족산', 'dongchundang': '동춘당', 'uineungjeongi_street': '으느정이문화의거리', 'ppuri_park': '뿌리공원', 'expo_science_park': '엑스포', 'sungsimdang_bakery': '성심당', 'water_barrel': '수통골', 'yuseong_hotspring': '유성온천', 'hanbat_arboretum': '한밭수목원', 'science_museum': '국립중앙과학관', 'daecheong_lake': '대청호', 'art_culture_complex': '대전문화예술단지', 'observatory': '시민천문대', 'oworld_zoo': '오월드'}

한국어 keyword => 영어 keyword
{'장태산': 'jangtae_mountain', '계족산': 'gyejok_mountain', '동춘당': 'dongchundang', '으느정이문화의거리': 'uineungjeongi_street', '뿌리공원': 'ppuri_park', '엑스포': 'expo_science_park', '성심당': 'sungsimdang_bakery', '수통골': 'water_barrel', '유성온천': 'yuseong_hotspring', '한밭수목원': 'hanbat_arboretum', '국립중앙과학관': 'science_museum', '대청호': 'daecheong_lake', '대전문화예술단지': 'art_culture_complex', '시민천문대': 'observatory', '오월드': 'oworld_zoo'}


## 대상 파일 설정

In [10]:
def concatCsv(data_type:str, keyword_kor='계족산'):
    '''
    정제 전 폴더 data와 정제 후 폴더 filtered간의 파일 구성이 따르기 때문에 
    주어진 폴더명에 따라 dataframe을 합해주는 함수
    '''

    if data_type == 'data':
        keyword_path = data_type + '/google_review/' + keyword_kor
        folder_path = os.path.join(g_data_path, keyword_path)
        print(f'folder name: {folder_path}')
        print(f'..: {os.listdir(folder_path)}')
        
        data_list = []
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            print(f'file path: {file_path}')

            df = pd.read_csv(file_path, encoding='utf-8') 
            search = file.split('.')[0]  # 파일명에서 파일형식(.csv) 제외 후, 검색어 추출             
            df['search'] = search
            df['keyword'] = kor2eng[keyword_kor] 
            data_list.append(df)
            print(f'해당 파일의 데이터 개수: {len(df)}')
            print()
        df = pd.concat(data_list, axis=0)
        print('-'*40)
        
    elif data_type == 'filtered':
        folder_path = os.path.join(g_data_path, data_type + '/google_review/' + target)
        print(f'folder name: {folder_path}')
        
        data_list = []    
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            print(f'file path: {file_path}')

            df = pd.read_csv(file_path, encoding='utf-8')  # csv 파일 읽기
            print(f'해당 파일의 데이터 개수: {len(df)}')
            print()
            data_list.append(df)
            print('-'*40)
            
        df = pd.concat(data_list, axis=0)
        print(target, '총 데이터 개수: ', len(df))
    return df

##  데이터 불러오기

In [11]:
data_df = concatCsv('filtered', _)

folder name: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420/google_reviews_daecheong_lake.csv
해당 파일의 데이터 개수: 491

----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420/google_reviews_sungsimdang_bakery.csv
해당 파일의 데이터 개수: 1863

----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420/google_reviews_water_barrel.csv
해당 파일의 데이터 개수: 685

----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_8420/google_reviews_observatory.csv
해당 파일의 데이터 개수: 26

----------------------------------------
file path: /home/aiffel-dj19/jungcheck/DataPreprocessing/filtered/google_review/labeled_spellcheck_

## 동반자 유형 사전 만들기 

In [12]:
companions = {
    '가족친지': ['남매', '자매', '형제', '가족 여행', '가족여행', '아기', '여보', '아내', '남편', '가족', '친척', '사촌', '부모님', '시부모', '장인어른', '아버님', '어머님', '조카', '이모', '고모', '삼촌', '할아버지', '할머니', '친형', '친언니', '친누나', '친동생', '친오빠', '엄마', '아빠', '딸', '아들', '자식'],
    '친구연인': ['동기', '후배', '우정 여행', '우정', '선배', '남자친구', '여자친구', '자기', '과동기', '찐친', '절친', '친구', '언니', '동생', '형', '오빠'],
    '직장학교': ['학과', '학교','수학여행', '현장견학', '현장체험', '직장', '회사', '주임님', '회사동료', '사장님', '직장동료', '동료', '직장동기', '회사동기', '회사', '직장선배', '직장후배', '선임', '후임', '부장', '부장님', '차장님', '대리님'],
    '단체': ['소풍', '단체'],
    '개인': ['혼자', '홀로', '개인']
}

In [13]:
companions.keys()

dict_keys(['가족친지', '친구연인', '직장학교', '단체', '개인'])

In [14]:
def dotAdd(sen):
    if sen[-1] in ['다', '요', '죠', '오', '자']:
        sen = sen + '.'
    return sen

## 품사 태깅을 위한 토크나이저 활용
* customized konlpy 활용
    - 장점: 커스텀된 단어의 품사를 추가할 수 있음 

In [15]:
tagger = Twitter()
tagger.add_dictionary(['남매', '자매', '형제', '아기', '가족', '친척', '사촌', '부모님', '시부모', '장인어른', 
                       '아버님', '어머님', '조카', '이모', '고모', '삼촌', '할아버지', '할머니', '친형', '친언니', 
                       '친누나', '친동생', '친오빠', '엄마', '아빠', '딸', '아들', '자식',
                      '동기', '후배', '선배', '남자친구', '여자친구', '자기', '과동기', '찐친', 
                       '절친', '친구', '언니', '동생', '형', '오빠',
                      '직장', '회사', '주임님', '회사동료', '사장님', '직장동료', '직장동기', '회사동기', 
                       '회사', '직장선배', '직장후배', '선임', '후임', '부장', '부장님', '차장님', '대리님',
                      '학과', '학교', '현장견학', '현장체험', '소풍', '수학여행', '주차장'], 'Noun')
tagger.add_dictionary(['랑', '와', '이와', '하고', '과 같이', '와 같이', '을', '를', '은', '는', '이', '가', '와 함께', '과 함께'], 'Josa')

In [16]:
postprocessor = Postprocessor(
    base_tagger = tagger, # base tagger
#     stopwords = ['랑', '와', '이와', '하고', '과 같이', '와 같이', '을', '를', '은', '는', '이', '가', '와 함께', '과 함께'], # 해당 단어 필터
#     passwords = passwords, # 해당 단어만 선택
    passtags = {'Noun', 'Josa', 'Verb'}, # 해당 품사만 선택
#     replace = replace, # 해당 단어 set 치환
#     ngrams = ngrams # 해당 복합 단어 set을 한 단어로 결합
)

## 품사태깅 테스트 

In [17]:
for word, pos in postprocessor.pos('아빠하고 아빠의 엄마랑 주차장을 차장님이랑  혼자 홀로 차장이랑 같이 왔다 갔다 방문했다'):
    print(word, pos)

아빠 Noun
하고 Josa
아빠 Noun
의 Josa
엄마 Noun
랑 Josa
주차장 Noun
을 Josa
차장님 Noun
이랑 Josa
혼자 Noun
홀로 Noun
차장 Noun
이랑 Josa
같이 Josa
왔다 Verb
갔다 Verb
방문 Noun
했다 Verb


In [18]:
postprocessor.pos(data_df['comment'].iloc[3123])

[('세', 'Noun'),
 ('이하', 'Noun'),
 ('아기', 'Noun'),
 ('랑', 'Josa'),
 ('가기', 'Noun'),
 ('넣', 'Verb'),
 ('무', 'Noun')]

In [19]:
'차장' == '주차장' # 주차장을 자꾸 차장으로 인식하는 오류 있음. 데이터 내에 차장과 관련된 데이터 없으므로 사전에서 제거 

False

In [20]:
from khaiii import KhaiiiApi

In [21]:
api = KhaiiiApi()

In [22]:
test_sen = data_df['comment'].iloc[3123]
data_df['comment'].iloc[3123]

'5세 이하 아기랑 가기 넣무 좋아요'

In [23]:
test_sen = '갔어요 갔다 갔고 가고 방문했고 방문하고 여행 갔고 오다 왔고 놀러 갔다 놀랐다 모시고 오다 시간을 보내다 곳이다'

In [24]:
morphs_dict = {}
for word in api.analyze(test_sen):
    for morph in word.morphs:
        morphs_dict[morph.lex] = morph.tag
#     print(word)
print(morphs_dict)

{'가': 'VV', '았': 'EP', '어요': 'EC', '다': 'EC', '고': 'EC', '방문': 'NNG', '하': 'XSV', '였': 'EP', '여행': 'NNG', '오': 'VV', '놀': 'VV', '러': 'EC', '놀라': 'VV', '모시': 'VV', '시간': 'NNG', '을': 'JKO', '보내': 'VV', '곳': 'NNG', '이': 'VCP'}


In [25]:
from konlpy.tag import Komoran

In [26]:
komoran = Komoran()

In [27]:
komoran.pos(test_sen)

[('가', 'VV'),
 ('았', 'EP'),
 ('어요', 'EC'),
 ('가', 'VV'),
 ('았', 'EP'),
 ('다', 'EC'),
 ('가', 'VV'),
 ('았', 'EP'),
 ('고', 'EC'),
 ('가', 'VV'),
 ('고', 'EC'),
 ('방문', 'NNG'),
 ('하', 'XSV'),
 ('았', 'EP'),
 ('고', 'EC'),
 ('방문', 'NNG'),
 ('하', 'XSV'),
 ('고', 'EC'),
 ('여행', 'NNG'),
 ('가', 'VV'),
 ('았', 'EP'),
 ('고', 'EC'),
 ('오', 'VV'),
 ('다', 'EC'),
 ('오', 'VX'),
 ('았', 'EP'),
 ('고', 'EC'),
 ('놀', 'VV'),
 ('러', 'EC'),
 ('가', 'VV'),
 ('았', 'EP'),
 ('다', 'EC'),
 ('놀라', 'VV'),
 ('았', 'EP'),
 ('다', 'EC'),
 ('모시', 'VV'),
 ('고', 'EC'),
 ('오다 시', 'NNP'),
 ('간', 'NNB'),
 ('을', 'JKO'),
 ('보내', 'VV'),
 ('다', 'EC'),
 ('곳', 'NNG'),
 ('이', 'VCP'),
 ('다', 'EC')]

=> Kahiii와 Komoran은 맞춤법이 지켜지지 않은 경우가 많은 데이터일 경우 성능이 좋은 토크나이저로 알려져 있습니다. 따라서 맞춤법 교정을 했음에도 구글 리뷰 특성상 오타가 있는 점을 감안하여, 해당 토크나이저 중 일반적으로 성능이 더 개선되어있다고 알려진 Kahiii를 적용했습니다.

## 데이터 중 하나로 테스트

In [28]:
print(data_df['comment'].iloc[3123])
for word, pos in postprocessor.pos(data_df['comment'].iloc[3123]):
    for key, value in companions.items():
    #     print(value)
        for v in value: 
            if word == v:
                print(word, '=>', v)

5세 이하 아기랑 가기 넣무 좋아요
아기 => 아기


## 각 관광지별로 동반자 유형 빈도수 추출 

In [29]:
import kss

In [30]:
test = '아'
test_lst = test + 'dkddk'
test_lst

'아dkddk'

In [31]:
[(word, pos) for word, pos in postprocessor.pos('확인해보자')]

[('확인', 'Noun'), ('해', 'Josa'), ('보자', 'Verb')]

In [38]:
def runAll(keyword):
    # 특정 관광지명을 대상으로 분석 
#     keyword = 'jangtae_mountain'
    test_df = data_df[data_df['keyword'] == keyword]
    test_list = test_df['comment']

    # 각 문장에 마침표 추가 후 한 문단으로 모든 문장을 합해준다. 
    test_list = list(map(dotAdd, test_list))
    test_list = ' '.join(test_list)
    sen_list = kss.split_sentences(test_list)
    
    josa_sen_list = []
    for sen in sen_list:        
        word_list = [(word, pos) for word, pos in postprocessor.pos(sen)]
        josa = ['랑', '을', '를', '은', '는', '이', '가', '와', '이와', '하고', '과 같이', '와 같이', '와 함께', '과 함께']
        isJosa = 0
        for word, pos in word_list:
#             print(word)
            if word in josa:
                isJosa = 1

        if isJosa == 1:
            josa_sen_list.append(sen)
#     print(josa_sen_list)
        
    # 방문자 유형을 확인할 수 있는 동사 리스트 
    # dict = lex:tag
    verb_dict = {'가':'VV', '놀':'VV', '오':'VV', '여행':'NNG', '방문':'NNG', '장소':'NNG', '곳':'NNG',
                 '모시':'VV', '보내':'VV'}
    verb_sen_list = []
    for sen in josa_sen_list:
        if len(sen) < 1: 
            continue
        else: 
            for word in api.analyze(sen):
                isVerb = 0
                for morph in word.morphs:
                    try: 
                        if morph.tag == verb_dict[morph.lex]:
                            isVerb = 1
                        else:
                            continue
                    except:
                        pass
            if isVerb == 1:
                verb_sen_list.append(sen)
#     print(verb_sen_list)

    # 동반자와 같이 올 수 있는 조사 형태가 있는 단어만 추출 
    companion_list = []
    for sen in verb_sen_list:
        isCompanion = 0
        key_list = []
        for word, _ in postprocessor.pos(sen):
            for key, value in companions.items():
                for v in value: 
                    if word == v:
                        print(word, '=>', key)  
                        key_list.append(key) # 동반자 유형을 할당 
                        isCompanion = 1
        if isCompanion == 1:
            companion_list = companion_list + list(set(key_list))
            print(sen)
            print()
#     test_list = ' '.join(sen_list)
    
    freq = Counter(companion_list)
    print(freq)
    
    if len(freq) != 0:
        companion_type = max(freq.keys(), key=(lambda k: freq[k]))
        print(companion_type)
        print()
    else: 
        print('동반자 유형을 알 수 없습니다.')
        print()
    
    return freq

In [39]:
for keyword in eng2kor.keys():
    key_kor = eng2kor[keyword]
    print('===========================관광지: ', keyword, '(', key_kor, ')', '============')
    runAll(keyword)
#     break

===========================관광지:  jangtae_mountain ( 장태산 ) ============
가족 => 가족친지
가족 및 아이들과 함께 시간을 보내기에 아름다운 곳입니다

소풍 => 단체
가깝고 조용하고 안전하고 길길 쭉쭉 메타세쿼이아 나무숲속 도시락 싸서 오손도손 소풍 오기 좋은 곳 도로도 점점 넓어져 오고 가는 길이 편해지는 곳 높다 하여 오르면 보이는 저수지의 놀라움 자연과 인공이 조화롭게 공존하는 곳입니다.

가족 => 가족친지
추석 연휴 가족과 함께 메타세쿼이아 숲길을 맘껏 누리고 갑니다.

가족 => 가족친지
나무가 정말 아름답습니다 가볼 만합니다. 가족단위로 방문하기 좋은 아주 좋은 곳입니다

가족 => 가족친지
시원하고 피톤치드 듬뿍 마시고 하늘 공원 빼어난 경치 외국에 와있는 듯한 수렵 한 번도 안 가본 사람은 있어도 한 번만 가본 사람은 없을 듯 깨끗하고 관리 잘 되어있는 편입니다. 가족과 산책하기 좋은 곳이죠.

Counter({'가족친지': 4, '단체': 1})
가족친지

===========================관광지:  gyejok_mountain ( 계족산 ) ============
가족 => 가족친지
경치도 좋고 황토 길음 맨발로 걷는 색다름 가족과 함께 걸으며 대화도 하고 황토의 차가움을 느끼며 오래 걸으면 건강에 도움 됨을 피부로 느낌 힐링하기 너무 좋은 장소입니다.

가족 => 가족친지
강추 시작점과 중간지점에 발을 씻을 수 있는 공간도 있어 수건 한 장 챙겨가면 좋을 듯 맨발 황토하고 나면 피로 확 풀림 걷기 좋은 곳 생각하기 좋은 곳 맨발로 걷고 가족끼리 오기 좋은 곳입니다.

가족 => 가족친지
계족산 계족산성 여름 가을 가족들과 맨발로 황톳길을 걸을 수 있어 좋은 곳입니다

Counter({'가족친지': 3})
가족친지

===========================관광지:  dongchundang ( 동춘당 ) ============
가족 => 가족친지
대전의 자랑스러운 

## 관광지 15개 종합하여 동반자 유형 빈도수 추출 

In [36]:
# 모든 관광지 중 방문자 유형 추출
test_list = data_df['comment']

# 각 문장에 마침표 추가 후 한 문단으로 모든 문장을 합해준다. 
test_list = list(map(dotAdd, test_list))
test_list = ' '.join(test_list)
sen_list = kss.split_sentences(test_list)

josa_sen_list = []
for sen in sen_list:        
    word_list = [(word, pos) for word, pos in postprocessor.pos(sen)]
    josa = ['랑', '을', '를', '은', '는', '이', '가', '와', '이와', '하고', '과 같이', '와 같이', '와 함께', '과 함께']
    isJosa = 0
    for word, pos in word_list:
#             print(word)
        if word in josa:
            isJosa = 1

    if isJosa == 1:
        josa_sen_list.append(sen)
#     print(josa_sen_list)

# 방문자 유형을 확인할 수 있는 동사 리스트 
# dict = lex:tag
verb_dict = {'가':'VV', '놀':'VV', '오':'VV', '여행':'NNG', '방문':'NNG', '장소':'NNG', '곳':'NNG',
             '모시':'VV', '보내':'VV'}
verb_sen_list = []
for sen in josa_sen_list:
    if len(sen) < 1: 
        continue
    else: 
        for word in api.analyze(sen):
            isVerb = 0
            for morph in word.morphs:
                try: 
                    if morph.tag == verb_dict[morph.lex]:
                        isVerb = 1
                    else:
                        continue
                except:
                    pass
        if isVerb == 1:
            verb_sen_list.append(sen)
#     print(verb_sen_list)

# 동반자와 같이 올 수 있는 조사 형태가 있는 단어만 추출 
companion_list = []
for sen in verb_sen_list:
    isCompanion = 0
    key_list = []
    for word, _ in postprocessor.pos(sen):
        for key, value in companions.items():
            for v in value: 
                if word == v:
                    print(word, '=>', key)  
                    key_list.append(key) # 동반자 유형을 할당 
                    isCompanion = 1
    if isCompanion == 1:
        companion_list = companion_list + list(set(key_list)) # 중복되는 key 제거
        print(sen)
        print()
#     test_list = ' '.join(sen_list)

freq = Counter(companion_list)
print(freq)

if len(freq) != 0:
    companion_type = max(freq.keys(), key=(lambda k: freq[k]))
    print(companion_type)
    print()
else: 
    print('동반자 유형을 알 수 없습니다.')
    print()

가족 => 가족친지
애들이나 노인을 동반한 가족 나들이로도 너무 좋고 연인이나 커플이 데이트하기에도 더없이 좋은 곳입니다

가족 => 가족친지
대청댐과 수자원에 대한 좋은 교육정보와 자료들이 잘 전시되어 있고 전망대에서는 멋진 주변 풍경을 감상할 수 있어서 가족들과 함께 나들이하며 휴식하기에 좋은 장소입니다.

가족 => 가족친지
지역 명소이기보단 지역에서 산책하고 나들이하기 좋은 곳 가족이 함께하기 좋은 장소입니다.

소풍 => 단체
단점은 조금 작은 규모와 편의시설 부재 촬영지 뭘 하기엔 너무 좁고 풍경도 별로 소풍 가기 좋은 곳이에요.

가족 => 가족친지
작은 연못에 산책로 덱 이 만들어져 있고 형형색색 가을 국화꽃이 즐비하고 향도 좋아 가족 나들이하기 좋은 곳입니다.

학교 => 직장학교
존 만 제가 세종 사는데 학교 시험 끝나거나 은행동 들릴 일 있으면 꼭 들리는 곳이에요

가족 => 가족친지
존 매트 주말이라 가족 연인들 손님들이 많고요 빵들을 몇 봉씩 많이 사 가요.

친구 => 친구연인
아주 다양한 빵과 여러 가지 종류의 음료도 함께 판매하고 전망 좋은 테이블 세팅이 있어 앉아서 친구 식구 지인들과 담소를 나누기 좋은 장소입니다.

가족 => 가족친지
운동을 싫어하는 나는 싫어하지만 아주 좋다 가족이 무척 애용한다 식당 되게 많고 뷰는 최고다 굳이 점수를 준다면 5점 만점에 4 325점 주말에 비가 와서 그런지 물도 맑고 피라미도 많고 시원한 바람 아주 굿 산책도 좋고 등산도 좋은 곳이에요

가족 => 가족친지
가족 => 가족친지
코로나 때문에 외출을 자재하고 있었는데 대전 동구 도담도담 지역아동센터에서 가족 초대를 해주어서 모처럼 만에 가족끼리 즐거운 시간을 보내고 왔습니다

가족 => 가족친지
경치도 좋고 황토 길음 맨발로 걷는 색다름 가족과 함께 걸으며 대화도 하고 황토의 차가움을 느끼며 오래 걸으면 건강에 도움 됨을 피부로 느낌 힐링하기 너무 좋은 장소입니다.

가족 => 가족친지
강추 시작점과 중간지점에 발을 씻을 수 있는 공간도 있어